# Data types and pattern matching

OCaml has a concise and expressive system for creating new datatypes. It also supports pattern matching to naturally express deconstruction of these data types.

## Type aliases

OCaml allows you to define aliases for existing types using the `type` keyword:

In [1]:
type int_pair = int * int

The above defines a type `int_pair` which is an alias to the pair of ints type (`int * int`).

You can constrain the type of an expression using the `:` operator. Here we create an identity function which is constrained to only work on pairs of ints:

In [2]:
let id x = (x : int_pair)

You can also constrain the type of a variable binding, so our `id` function could also be written as:

In [3]:
let id (x : int_pair) = x

## Records

### Record types

Records in OCaml represent a collection of named elements. A simple example is a `point` record containing `x`, `y` and `z` fields:

In [4]:
type point = {
  x : int;
  y : int;
  z : int;
}

We can create instances of our `point` type using `{ ... }`, and access the elements of a point using the '.' operator:

In [5]:
let origin = { x = 0; y = 0; z = 0 }

let get_y r = r.y

let o = get_y origin

### Functional update

New records can also be created from existing records using the `with` keyword. For example, we can create a new `point` which is the same as `origin` except with the value of its `z` field changed to `10`:

In [6]:
let p = { origin with z = 10 }

### Field punning

Another useful trick with records is *field punning*, which allows you to replace:

In [7]:
let mk_point x y z = { x = x; y = y; z = z }

with

In [8]:
let mk_point x y z = { x; y; z }

## Mutability

OCaml has first-class support for mutability. There are several langauge features that support mutability in OCaml. One of them is mutable record fields. Let us make a mutable point datatype.

In [9]:
type mpoint = {mutable x : int; mutable y : int; mutable z: int}

Notice that the type says that the fields are `mutable`. Just like the point datatype defined previously, you can create a value of `mpoint` type and read it.

In [10]:
let morigin = {x=0;y=0;z=0}

let p = {morigin with z = 10}

let p_z = p.z

However, unlike the immutable record fields, the mutable fields can be updated.

In [11]:
p.z <- 20

let p_z = p.z

### References

It is sometimes useful to create single mutable value. OCaml provides reference cells for this purpose

In [12]:
let x = ref 0

`x` is a reference cell which holds a value of type integer and its current value is 0. Reference cells can be read using `!` and updated using `:=`.

In [13]:
x := !x + 1

let v = !x

<h3> <span style="color:purple;border-style:solid"> Exercise </span> </h3>

Implement a function that takes two mutable variable and swaps their values:

In [14]:
let swap x y = failwith "for you to implement"

In [15]:
let x = ref 10
let y = ref 20

let () = swap x y

error: runtime_error

In [16]:
assert ((20,10) = (!x,!y))

error: runtime_error

## Variants

### Variant types

Variants in OCaml represent data which can be in one of a number of forms. A very simple example is a type representing one of three colours:

In [17]:
type colour =
  | Red
  | Green
  | Blue

type colour = Red | Green | Blue


We can create a `colour` using one of its constructors:

In [18]:
let red = Red

val red : colour = Red


### Constructor arguments

Variant constructors can also have arguments. This allows variants to contain different types of data depending on which constructor was used. For example, we can create a type which contains either a `point` or a `colour`:

In [19]:
type t =
  | Point of point
  | Colour of colour
  
let p_or_c cond pnt col = if cond then Point pnt else Colour col

let p = p_or_c (1 > 0) origin red

type t = Point of point | Colour of colour


val p_or_c : bool -> point -> colour -> t = <fun>


val p : t = Point {x = 0; y = 0; z = 0}


### Multiple constructor arguments

Variants constructors can contain multiple arguments seperated by the `*` symbol:

In [20]:
type s =
| ThreePoints of point * point * point
| TwoColours of colour * colour

type s = ThreePoints of point * point * point | TwoColours of colour * colour


Creating these constructors with multiple arguments require parentheses:

In [21]:
let s = TwoColours(Red, Green)

val s : s = TwoColours (Red, Green)


## Pattern matching

Before we go on, let us define a handy print function to print stuff to the notebook. 

In [66]:
#require "jupyter.notebook";;

let show s = ignore (Jupyter_notebook.display "text/html" ("<h3 style='color:red'>"^s^"</h3>"))

val show : string -> unit = <fun>


Now we can print things to screen

In [67]:
show "Hello, world!"

Hello, world!

- : unit = ()


### Inspecting variants

So far we have created some values of variant types, but how do we get the data back out of them? The answer is *pattern matching*. Using a `match` statement we can deconstruct a variant type and retrieve its constructor's arguments:

In [68]:
let print_t t =
  match t with
  | Point p -> show (Printf.sprintf "Point: %d %d %d\n" p.x p.y p.z)
  | Colour c -> show (Printf.sprintf "Colour\n")
  
let () = print_t (Point { x = 5; y = 9; z = 0 })

let () = print_t (Colour Blue)

val print_t : t -> unit = <fun>


Point: 5 9 0

Colour

Here `Point p` and `Colour c` are not expressions but *patterns*. They describe the shape of the data and bind variables to different parts of it. Note that the `p` in `Point p` does not refer to an existing `p` variable, instead it is creating a new `p` variable bound to the argument of the `Point` constructor.

### Nested patterns

We can nest patterns within other patterns to do pattern matching on the constructor arguments. For example, we can print the names of the different colours in our `print_t` function:

In [70]:
let print_t t =
  match t with
  | Point p -> show (Printf.sprintf "Point: %d %d %d\n" p.x p.y p.z)
  | Colour Red -> show (Printf.sprintf "Red\n")
  | Colour Green -> show (Printf.sprintf "Green\n")
  | Colour Blue -> show (Printf.sprintf "Blue\n")
  
let () = print_t (Colour Red)

let () = print_t (Colour Blue)

val print_t : t -> unit = <fun>


Red

Blue

### Matching records

We can also match on record data using the same syntax as to create records, including field punning. So our `print_t` can be further refined to:

In [71]:
let print_t t =
  match t with
  | Point { x; y; z } -> Printf.printf "Point: %d %d %d\n" x y z
  | Colour Red -> Printf.printf "Red\n"
  | Colour Green -> Printf.printf "Green\n"
  | Colour Blue -> Printf.printf "Blue\n"

val print_t : t -> unit = <fun>


### Exhaustiveness

A key feature of pattern matching, which can help prevent many errors especially when refactoring, is that the compiler will warn you if you forget to handle a particular case. For example, if we had forgotten the `Colour Green` case in the above definition:

In [84]:
let print_t_ t =
  match t with
  | Point { x; y; z } -> show (Printf.sprintf "Point: %d %d %d\n" x y z)
  | Colour Red -> show (Printf.sprintf "Red\n")
  | Colour Blue -> show (Printf.sprintf "Blue\n")

File "[84]", line 2, characters 2-185:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Colour Green


val print_t_ : t -> unit = <fun>


### The `_` pattern

Sometimes, you do not care about the value of a constructor, in this case you can use the `_` pattern which will match any constructor:

In [85]:
let is_colour_red t =
  match t with
  | Colour Red -> true
  | _ -> false

val is_colour_red : t -> bool = <fun>


### Match ordering

Note that patterns are matched from top to bottom: if a value matches multiple patterns then the first of those patterns will be selected. For example, in the following code the second case will never be matched:

In [86]:
let is_colour_red t =
  match t with
  | _ -> false
  | Colour Red -> true

File "[86]", line 4, characters 4-14:
Warning 11: this match case is unused.


val is_colour_red : t -> bool = <fun>


## Parameterised types

Types in OCaml can be parameterised by other types. For example, the `option` type which may or may not contain a value:

In [87]:
type 'a option =
| None
| Some of 'a

type 'a option = None | Some of 'a


In the above the `'a` is a *type variable*, which can be substituted by any type. For instance a we can create a value of type `int option` or a value of type `colour option`:

In [88]:
let io = Some 6

let co = Some Green

val io : int option = Some 6


val co : colour option = Some Green


We can define a printer for `t option` type as follows:

In [91]:
let print_t_opt t = 
  match t with
  | None -> show ("None")
  | Some t -> print_t t

val print_t_opt : t option -> unit = <fun>


In [97]:
let () = print_t_opt (Some (Colour Red))

let () = print_t_opt None

Red

None

### Polymorphic values

These type variables also appear when creating *polymorphic* values. For example, the following function has type `'a option -> 'a list` which means it can be applied to any `option` type:

In [98]:
let opt_to_list o =
  match o with
  | Some x -> [x]
  | None -> []
  
let l = opt_to_list (Some 9)

let m = opt_to_list (Some Red)

val opt_to_list : 'a option -> 'a list = <fun>


val l : int list = [9]


val m : colour list = [Red]


### Polymorphic constructors

Constructors of parameterised types which do not include the type parameter -- such as `None` in the optional type -- are also examples of polymorphic values:

In [99]:
let n = None

let a = [ Some 3; n ]

let b = [ n; Some Blue ]

val n : 'a option = None


val a : int option list = [Some 3; None]


val b : colour option list = [None; Some Blue]


## Recursive data types

Data types in OCaml can also be recursive. This allows us to create recursive structures such as trees and lists. The following defines a parametric binary tree type:

In [100]:
type 'a binary_tree =
  | Leaf of 'a
  | Tree of 'a binary_tree * 'a binary_tree

type 'a binary_tree = Leaf of 'a | Tree of 'a binary_tree * 'a binary_tree


As you can see the `Tree` constructor of `binary_tree` contains other `binary_tree`s as its arguments.

### Inspecting recursive data types

We can write recursive functions to handle these recursive data types. For example, the following function returns the maximum depth of a binary tree:

In [101]:
let rec depth tr =
  match tr with
  | Leaf _ -> 1
  | Tree(left, right) ->
      1 + (max (depth left) (depth right))

let tree = 
  Tree(Tree(Leaf Red,
            Tree(Leaf Blue,
                 Leaf Green)),
       Tree(Leaf Green,
            Leaf Green))

let d = depth tree

val depth : 'a binary_tree -> int = <fun>


val tree : colour binary_tree =
  Tree (Tree (Leaf Red, Tree (Leaf Blue, Leaf Green)),
   Tree (Leaf Green, Leaf Green))


val d : int = 4


## Lists

### Constructing lists

A particularly common built-in data type in OCaml is the `list` type. `list` is actually a parameterised recursive variant type. It has two constructors `::` (called cons) and `[]` (called nil). `[]` represents an empty list and `::` adds an element to the front of the list:

In [35]:
let l = 1 :: 2 :: 3 :: []

val l : int list = [1; 2; 3]


OCaml also provides a short-hand syntax for lists: `[ ..; .. ]`. Our `l` value above could instead have been defined:

In [36]:
let l = [1; 2; 3]

val l : int list = [1; 2; 3]


### Matching lists

Like all constructors, the list constructors can be used as patterns in pattern matching. The following function sums all the elements of an `int list`:

In [37]:
let rec sum il =
  match il with
  | [] -> 0
  | i :: rest -> i + (sum rest)
  
let s = sum l

val sum : int list -> int = <fun>


val s : int = 6


<h3> <span style="color:purple;border-style:solid"> Exercise </span> </h3>

Write a function `min_list` to compute the minimum element in an integer list. If the list is empty then it should return `None`. If the minimum element is `e`, then the function returns `Some e`.

In [107]:
let rec min_list_helper cur_min l =
  match l with
  | [] -> cur_min
  | x::xs ->
      match cur_min with
      | None -> failwith "for you to implement"
      | Some m -> failwith "for you to implement"
  
let min_list l = min_list_helper None l

val min_list_helper : 'a option -> 'b list -> 'a option = <fun>


val min_list : 'a list -> 'b option = <fun>


In [108]:
assert (min_list [] = None)

- : unit = ()


In [109]:
assert (min_list [3;1;2] = Some 1)

error: runtime_error